In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd
dataset = load_dataset("ai4privacy/pii-masking-300k")
train = load_dataset("ai4privacy/pii-masking-300k", split = "train")
test = load_dataset("ai4privacy/pii-masking-300k", split = "validation")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train/1english_openpii_30k.jsonl:   0%|          | 0.00/103M [00:00<?, ?B/s]

data/train/dutch_openpii_28k.jsonl:   0%|          | 0.00/102M [00:00<?, ?B/s]

data/train/french_openpii_31k.jsonl:   0%|          | 0.00/114M [00:00<?, ?B/s]

...

data/validation/german_openpii_8k.jsonl:   0%|          | 0.00/29.2M [00:00<?, ?B/s]

data/validation/italian_openpiii_8k.json(…):   0%|          | 0.00/28.3M [00:00<?, ?B/s]

data/validation/spanish_openpii_8k.jsonl:   0%|          | 0.00/27.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177677 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47728 [00:00<?, ? examples/s]

In [ ]:
# basic information
print(dataset)
print(train)
print(test)
print(train.column_names)

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})
Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 177677
})
Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 47728
})
['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set']


In [ ]:
# looking at features for training data and removing any data that isn't of the English language
print(train.features)
print(train.unique("language"))
train = train.filter(lambda x: x["language"] == "English")
print(train)
print(train["language"][:10])

{'source_text': Value('string'), 'target_text': Value('string'), 'privacy_mask': List({'value': Value('string'), 'start': Value('int64'), 'end': Value('int64'), 'label': Value('string')}), 'span_labels': Value('string'), 'mbert_text_tokens': List(Value('string')), 'mbert_bio_labels': List(Value('string')), 'id': Value('string'), 'language': Value('string'), 'set': Value('string')}
['English', 'Dutch', 'French', 'German', 'Italian', 'Spanish']


Filter:   0%|          | 0/177677 [00:00<?, ? examples/s]

Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 29908
})
['English', 'English', 'English', 'English', 'English', 'English', 'English', 'English', 'English', 'English']


In [ ]:
# basic info
# length of training went from 177677 to 29908
print(train.unique("language"))
print(train)

Flattening the indices:   0%|          | 0/29908 [00:00<?, ? examples/s]

['English']
Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 29908
})


In [ ]:
# looking at features for training data and removing any data that isn't of the English language
print(test.features)
print(test.unique("language"))
test = test.filter(lambda x: x["language"] == "English")

{'source_text': Value('string'), 'target_text': Value('string'), 'privacy_mask': List({'value': Value('string'), 'start': Value('int64'), 'end': Value('int64'), 'label': Value('string')}), 'span_labels': Value('string'), 'mbert_text_tokens': List(Value('string')), 'mbert_bio_labels': List(Value('string')), 'id': Value('string'), 'language': Value('string'), 'set': Value('string')}
['English', 'Dutch', 'French', 'German', 'Italian', 'Spanish']


Filter:   0%|          | 0/47728 [00:00<?, ? examples/s]

In [ ]:
# basic info
# length of testing went from 47728 to 7946
print(test.unique("language"))
print(test)

Flattening the indices:   0%|          | 0/7946 [00:00<?, ? examples/s]

['English']
Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
    num_rows: 7946
})


In [ ]:
# for manually inspecting a few sample datapoints
for i in range(5):
    sample = train[i]
    print("Source:", sample["source_text"])
    print("Spans:", sample["privacy_mask"], sample["span_labels"])
    print("Target:", sample["target_text"])
    print("\n---------------------------------\n")

Source: Subject: Group Messaging for Admissions Process

Good morning, everyone,

I hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:

- wynqvrh053 - Meeting at 10:20am
- luka.burg - Meeting at 21
- qahil.wittauer - Meeting at quarter past 13
- gholamhossein.ruschke - Meeting at 9:47 PM
- pdmjrsyoz1460 
Spans: [{'value': 'wynqvrh053', 'start': 287, 'end': 297, 'label': 'USERNAME'}, {'value': '10:20am', 'start': 311, 'end': 318, 'label': 'TIME'}, {'value': 'luka.burg', 'start': 321, 'end': 330, 'label': 'USERNAME'}, {'value': '21', 'start': 344, 'end': 346, 'label': 'TIME'}, {'value': 'qahil.wittauer', 'start': 349, 'end': 363, 'label': 'USERNAME'}, {'value': 'quarter past 13', 'start': 377, 'end': 392, 'label': 'TIME'}, {'value': 'gholamhossein.ruschke', 'start': 395, 'end': 416, 'label': 'USERNAME'}, {'value': '9:47 PM', 'start':

In [ ]:
train_df = pd.DataFrame(train)
train_df.head()

test_df = pd.DataFrame(test)
test_df.head()

,source_text,target_text,privacy_mask,span_labels,mbert_text_tokens,mbert_bio_labels,id,language,set
0,On the video sharing platform for educational ...,On the video sharing platform for educational ...,"[{'value': 'paaltwvkjuijwbj957', 'start': 165,...","[[316, 320, ""USERNAME""], [254, 272, ""USERNAME""...","[On, the, video, sharing, platform, for, educa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40770A,English,validation
1,d wsfdkmi9214 shared personal experiences rela...,d [USERNAME] shared personal experiences relat...,"[{'value': 'wsfdkmi9214', 'start': 2, 'end': 1...","[[355, 356, ""USERNAME""], [339, 350, ""USERNAME""...","[d, w, ##s, ##f, ##d, ##km, ##i, ##9, ##21, ##...","[O, B-USERNAME, I-USERNAME, I-USERNAME, I-USER...",40770B,English,validation
2,ach other respectfully in their comments. yega...,ach other respectfully in their comments. [USE...,"[{'value': 'yeganeh-afchar', 'start': 42, 'end...","[[339, 344, ""TIME""], [61, 74, ""USERNAME""], [42...","[ach, other, respect, ##fully, in, their, comm...","[O, O, O, O, O, O, O, O, B-USERNAME, I-USERNAM...",40770C,English,validation
3,"{\n ""Guidelines"": {\n ""maximumAge"": ""May 5...","{\n ""Guidelines"": {\n ""maximumAge"": ""[BOD]...","[{'value': 'May 5th, 1966', 'start': 39, 'end'...","[[432, 441, ""IDCARD""], [391, 400, ""IDCARD""], [...","[{, "", Guide, ##lines, "", :, {, "", maximum, ##...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40771,English,validation
4,Subject: Collaboration Opportunity in Online C...,Subject: Collaboration Opportunity in Online C...,[],[],"[Sub, ##ject, :, Coll, ##aboration, Op, ##port...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40779A,English,validation


In [ ]:
# Random Forest Classifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


train_df["is_sensitive"] = train_df["privacy_mask"].apply(lambda x: 1 if len(x) > 0 else 0)
test_df["is_sensitive"] = test_df["privacy_mask"].apply(lambda x: 1 if len(x) > 0 else 0)

# now we have a clean dataframe
# print(train_df.head(25))
print(train_df[["language", "is_sensitive"]].value_counts())


# vectorize text with TF-IDF
vectorizer = TfidfVectorizer(
    max_features=1500,
    stop_words="english"
)

X_train = vectorizer.fit_transform(train_df["source_text"])
X_test = vectorizer.transform(test_df["source_text"])
y_train = train_df["is_sensitive"]
y_test = test_df["is_sensitive"]


# train random forest classifier
rf = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)

# Best parameters overall, but context here matters
# With sensitive data, it is paramount that we have near 100% recall
# This random forest below unfortunately has a recall < .9
# rf = RandomForestClassifier(
#     n_estimators = 404,
#     max_depth = 48,
#     min_samples_split = 3,
#     min_samples_leaf = 3,
#     max_features = 'sqrt',
#     n_jobs=-1,
#     random_state=42,
#     class_weight='balanced'
# )

rf.fit(X_train, y_train)


# evaluate
y_pred = rf.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))


feature_names = vectorizer.get_feature_names_out()
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1][:20]

print("\nTop 20 Important Words for Sensitive Detection:")
for i in indices:
    print(f"{feature_names[i]}: {importances[i]:.4f}")

language  is_sensitive
English   1               26426
          0                3482
Name: count, dtype: int64
Confusion Matrix:
[[ 805  103]
 [ 956 6082]]

Classification Report:
              precision    recall  f1-score   support

           0      0.457     0.887     0.603       908
           1      0.983     0.864     0.920      7038

    accuracy                          0.867      7946
   macro avg      0.720     0.875     0.762      7946
weighted avg      0.923     0.867     0.884      7946


Top 20 Important Words for Sensitive Detection:
com: 0.0512
number: 0.0332
passport: 0.0292
username: 0.0243
id: 0.0227
address: 0.0193
road: 0.0187
ip: 0.0187
11: 0.0186
date: 0.0180
00: 0.0177
eng: 0.0176
pm: 0.0156
license: 0.0146
password: 0.0130
street: 0.0124
time: 0.0123
sex: 0.0122
12: 0.0120
email: 0.0117


In [ ]:
# pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 21.6 MB/s eta 0:00:00


In [ ]:
# Takes like 1.5 hours to run
# Best Params: {'n_estimators': 404, 'max_depth': 48, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 'sqrt'}
# Best F1 Score: 0.918767248955558


# import optuna
# from sklearn.model_selection import cross_val_score

# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#         'max_depth': trial.suggest_int('max_depth', 5, 50),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
#         'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
#         'class_weight': 'balanced',
#         'random_state': 42
#     }
#     rf = RandomForestClassifier(**params)
#     f1 = cross_val_score(rf, X_train, y_train, scoring='f1', cv=3, n_jobs=-1).mean()
#     return f1

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=30)

# print("Best Params:", study.best_params)
# print("Best F1 Score:", study.best_value)







# best_rf = random_search.best_estimator_  # or grid_search.best_estimator_
# best_rf.fit(X_train, y_train)
# y_pred = best_rf.predict(X_test)
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred, digits=3))

[I 2025-10-16 05:22:07,422] A new study created in memory with name: no-name-84c90b0f-7c5f-46a8-a078-3cfad794a7f1
[I 2025-10-16 05:22:37,721] Trial 0 finished with value: 0.9058125108434226 and parameters: {'n_estimators': 196, 'max_depth': 29, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9058125108434226.
[I 2025-10-16 05:22:50,907] Trial 1 finished with value: 0.9064922805993144 and parameters: {'n_estimators': 196, 'max_depth': 38, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 1 with value: 0.9064922805993144.
[I 2025-10-16 05:23:14,801] Trial 2 finished with value: 0.8661074518606462 and parameters: {'n_estimators': 336, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9064922805993144.
[I 2025-10-16 05:47:40,980] Trial 3 finished with value: 0.9134530546620256 and parameters: {'n_estimators': 434, 'max_depth': 41, 'm

Best Params: {'n_estimators': 404, 'max_depth': 48, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 'sqrt'}
Best F1 Score: 0.918767248955558


NameError: name 'random_search' is not defined